In [48]:
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import configparser
from datetime import date
import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from datetime import date
import pickle
import yfinance as yf
import ast
from clickhouse_driver import Client


pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', None)

#config.read('/Users/p.matchenkov/NotebookProjects/configurations/config.ini')

In [49]:
def connect_click_mybro():
    config = configparser.ConfigParser()
    config.read('/Users/p.matchenkov/Desktop/configurations/config.ini')
    password = config['MYBRO_CEX_clickhouse']['password']
    login = config['MYBRO_CEX_clickhouse']['login']
    localhost = config['MYBRO_CEX_clickhouse']['localhost']
    database = config['MYBRO_CEX_clickhouse']['database']
    
    client = Client(localhost,
                    user=login,
                    password=password,
                    database=database)
    return client


def get_market_data():
    req = f"""
        select prodbrokerdb.market_data.symbol as quote_currency, prodbrokerdb.market_data.date_added as executed_date,
        case when prodbrokerdb.market_data.symbol = 'USD/JPY' then (1/close)
            else close
        end as close
        from prodbrokerdb.market_data md 
        where prodbrokerdb.market_data.symbol in ('BTC/USD', 'ETH/USD', 'GBP/USD', 'EUR/USD', 'USD/JPY')
        and prodbrokerdb.market_data.candle = 'h'
    """

    df, columns = engine.execute(req, with_column_types=True)
    df = pd.DataFrame(df, columns=[tuple[0] for tuple in columns]).drop_duplicates(['quote_currency', 'executed_date'])
    df['executed_date'] = pd.to_datetime(df['executed_date'])
    df['quote_currency'] = df['quote_currency'].apply(lambda x: x.replace('/USD', ''))
    df['quote_currency'] = df['quote_currency'].apply(lambda x: x.replace('USD/', ''))
    return df

def avg_trade_time(orders):
    df = (orders[['user_id', 'account_id', 'executed_date_time', 'position_code']]
          .loc[orders['position_effect'] == 'OPENING']
          .merge(orders[['executed_date_time', 'position_code']].loc[orders['position_effect'] == 'CLOSING'],
                  how='left', on = 'position_code'))
    
    df['time_open'], df['time_close'] = pd.to_datetime(df['executed_date_time_x']), pd.to_datetime(df['executed_date_time_y'])

    df['trade_time'] = (df['time_close'] - df['time_open'])#.total_seconds()
    df['trade_time'] = df['trade_time'] / np.timedelta64(1, 'h')
    df = df.drop(columns={'executed_date_time_x', 'position_code', 'executed_date_time_y'})
    
    df = (df.groupby('user_id')
          .agg({'trade_time': 'mean'})
          .reset_index()
          .rename(columns={'time_close': 'date'}))
    return df

def get_financ_transc(df): # поставить в first period data
    btc = yf.download('BTC-USD', start='2019-01-01') 
    eth = yf.download('ETH-USD', start='2019-01-01')
    btc['quote_currency'] = 'BTC'
    eth['quote_currency'] = 'ETH'
    market_data = pd.concat([btc[['Close', 'quote_currency']], eth[['Close', 'quote_currency']]], axis=0)
    market_data.columns = ['close', 'quote_currency'] 
    market_data.index.names = ['executed_date']
    market_data = market_data.reset_index()
    del btc, eth

    df['first_trade'] = pd.to_datetime(df['first_trade'])
    request = f"""
        select user_id, symbol as quote_currency, record_type, total_amount, date_time_added, date_added as executed_date
        from prodbrokerdb.finance_statistics_hour 
        where prodbrokerdb.finance_statistics_hour.record_type in ('deposit', 'withdrawal')
        and user_id != 'up150483576'
        order by prodbrokerdb.finance_statistics_hour.date_time_added
    """
    engine = connect_click_mybro()
    deps, columns = engine.execute(request, with_column_types=True)
    deps = pd.DataFrame(deps, columns=[tuple[0] for tuple in columns]).drop_duplicates()
    deps['executed_date'] = pd.to_datetime(deps['executed_date'])
    
    # merging with md
    deps = (deps.merge(market_data, how='left', on=['executed_date', 'quote_currency'])
                .merge(df[['user_id', 'first_trade']], how='left', on='user_id'))

    
    deps['close'] = deps['close'].fillna(1)
    deps['deposit'] = deps['total_amount'].loc[deps['record_type'] == 'deposit'] * deps['close']
    deps['withdrawal'] = deps['total_amount'].loc[deps['record_type'] == 'withdrawal'] * deps['close']
    deps['date_time_added'] = pd.to_datetime(deps['executed_date'])
    deps['days_diff'] = (pd.to_datetime(deps['date_time_added']) - pd.to_datetime(deps['first_trade'])).dt.days # diff days between trans time and first trade
    deps = deps.loc[(deps['days_diff'] <= 14)] # delete all trans after 14 days

    
    deps = (deps.groupby('user_id')
            .agg({'deposit': 'sum', 'withdrawal': 'sum'})
            .merge(
                deps.loc[deps['record_type']=='deposit']
                .groupby('user_id').agg({'date_time_added': 'min', 'deposit': 'first'})
                .rename(columns={'date_time_added': 'first_dep_date', 'deposit': 'first_dep'}), how='outer', on='user_id'
            )
            .merge(
                deps.loc[deps['record_type']=='withdrawal']
                .groupby('user_id').agg({'date_time_added': 'min', 'withdrawal': 'first'})
                .rename(columns={'date_time_added': 'first_with_date', 'withdrawal': 'first_withd'}), how='outer', on='user_id'
            ))
    deps['first_dep_date'] = pd.to_datetime(deps['first_dep_date'].dt.date)
    deps['first_with_date'] = pd.to_datetime(deps['first_with_date'].dt.date)
    return deps



In [50]:
forex = ["USD/JPY","EUR/GBP","EUR/JPY","EUR/USD","GBP/JPY","GBP/USD","XAG/USD","XAU/USD", "USD/RUB"]
stocks = ["AAPL", "AMZN","META","TSLA","SPC","NFLX","GOOG"]

_internal_users = ['Compliance', 'up150483576']

In [51]:
engine = connect_click_mybro()
req = f"""
        select user_id, account_id, order_strategy, order_symbol, executed_date_time, position_code,
        price, quantity, quote_currency, executed_date, order_pl, markup, position_effect, user_created_date_time
        from prodbrokerdb.order_statistics 
        where account_clearing_code = 'live' and jurisdiction = 'SVT'
        and executed_date_time >= '2020-01-01'
        order by executed_date_time
        """
orders, columns = engine.execute(req, with_column_types=True)
orders = (pd.DataFrame(orders, columns=[tuple[0] for tuple in columns])
      .drop_duplicates().replace('USDT', 'USD')
      .replace('', np.nan)
      .fillna({'quote_currency': 'USD'}))

orders = (orders.loc[
        (~(orders['user_id'].isin(_internal_users))) 
        #&
        #(~orders['order_symbol'].isin(stocks+forex))
        ])

market_data = get_market_data()

In [52]:
len(orders['user_id'].unique())

3190

deop

In [53]:
# orders['executed_date'] = pd.to_datetime(orders['executed_date'])
# orders['quote_currency'] = orders['order_symbol'].str.extract('/(.*)')


# orders = orders.merge(market_data[['executed_date', 'quote_currency', 'close']], how='left', on=['executed_date', 'quote_currency'])


# orders['close'] = orders['close'].replace('', np.nan).fillna(1)
# orders['volume'] = orders['quantity'] * orders['price'] * orders['close']
# orders['pnl'] = orders['order_pl'] * orders['close']

# orders['executed_date'] = pd.to_datetime(orders['executed_date'])
# orders = orders.loc[orders['order_symbol']=='XAU/USD'].groupby('user_id').agg({'volume': sum, 'pnl': sum})#.groupby([pd.Grouper(key='executed_date', how='left', freq='W-MON')]).agg({'volume': sum, 'pnl': sum})
# orders['pnl/vol'] = orders['pnl'] / orders['volume'] * 10**6
# orders

In [54]:
orders['executed_date'] = pd.to_datetime(orders['executed_date'])
orders['quote_currency'] = orders['order_symbol'].str.extract('/(.*)')


total = orders.merge(market_data[['executed_date', 'quote_currency', 'close']], how='left', on=['executed_date', 'quote_currency'])


total['close'] = total['close'].replace('', np.nan).fillna(1)
total['volume'] = total['quantity'] * total['price'] * total['close']
total['pnl'] = total['order_pl'] * total['close']


first_trade = total.groupby('user_id').agg({'executed_date': 'min'}).reset_index()
total = total.merge(first_trade, how='left', on='user_id').rename(columns={'executed_date_x': 'trade_date', 'executed_date_y': 'first_trade'})
del first_trade


total['user_created_date_time'] = pd.to_datetime(total['user_created_date_time']).dt.date
total['days_diff'] = (pd.to_datetime(total['trade_date']) - pd.to_datetime(total['first_trade'])).dt.days
__total = total.copy() # save full trading history for full pnl and VOLUME column
total = total.loc[(total['days_diff'] <= 14)]# &
total_14days = total.copy() # for avg_trade_time in 14 days period

# creatin leverage df
# accounts = list(total['account_id'].unique())
# levereges = get_account_leverage(accounts=accounts)
# levereges['leverage'] = levereges['name'].str.extract('\:(.*)').astype(int)
# levereges['user_id'] = levereges['account_id'].str.extract('_(.*)_')
# levereges = levereges[['account_id', 'leverage']]
# # concat with leverage value
# total = total.merge(levereges[['account_id', 'leverage']], how='left', on='account_id')


total = (
        total
        .groupby('user_id')
        .agg({'pnl': sum, 
              'volume': ['sum', 'mean', 'median'],
              'user_id': 'count',
              'account_id': pd.Series.nunique,
              'first_trade': 'first',
              'user_created_date_time': 'min'
              })
        .reset_index()
        )
total.columns = ['user_id', 'pnl', 'vol_sum', 'vol_mean', 'vol_median', 'trades_qty', 'account_qty', 'first_trade', 'user_created_date_time'] #, 'lev_sum'
total = total.merge(avg_trade_time(total_14days), how='left', on='user_id')
del total_14days

print(len(total['user_id'].unique()))   
display(get_financ_transc(total).tail())
# merge with deps_withd info
total = total.merge(get_financ_transc(total), how='left', on='user_id')
        

3190
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,deposit,withdrawal,first_dep_date,first_dep,first_with_date,first_withd
user_id,,,,,,
ws2014,"2,564.96",409.79,2020-01-10,"1,004.48",2020-02-18,104.32
xpk,"1,293.15","2,202.53",2020-10-23,"1,293.15",2020-10-23,538.84
xwerewolf,156.58,0.00,2020-04-22,156.58,NaT,NaN
zavodchik,143.71,124.87,2020-04-16,7.12,2020-05-06,7.73
zx007,302.85,191.50,2020-04-22,2.85,2021-01-27,191.50


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [55]:
# creating features
total['withd_cond'] = (total['first_with_date'] - total['first_trade']).dt.days 
#total = total.drop(total.loc[total['withd_cond']<0].index)
total.loc[total['withd_cond'] > 14, 'withd_cond'] = 0
total.loc[total['withd_cond'] <= 14, 'withd_cond'] = 1 # был ли вывод в первые 14 дней после первой сделки
total['dif_dep_with'] = total['first_dep'] - total['first_withd']
total['dif_dep_with'] = total['dif_dep_with'].fillna(0)

print(len(total['user_id'].unique()))   

total['user_created_date_time'] = pd.to_datetime(total['user_created_date_time']) # начал ли торговлю в теч 14 дней после регитсрации
total['first_trade'] = pd.to_datetime(total['first_trade'])
total['diff_reg_trade'] = (total['user_created_date_time'] - total['first_trade']).dt.days
total.loc[total['diff_reg_trade'] > 14, 'diff_reg_trade'] = 0
total.loc[total['diff_reg_trade'] <= 14, 'diff_reg_trade'] = 1
total = total.drop(columns={'first_trade', 'first_dep_date', 'first_with_date'})

market_data['user_created_date_time'] = market_data['executed_date']
market_data = market_data.loc[market_data['quote_currency']=='BTC']

# btc price to datafrme
msft = yf.Ticker("BTC-USD")
hist = msft.history(start="2018-01-01").reset_index()
hist['user_created_date_time'] = pd.to_datetime(pd.to_datetime(hist['Date']).dt.date)
hist = hist[['user_created_date_time', 'Close']]
total = (total.merge(
    hist, how='left', on='user_created_date_time'
        ).rename(columns={'Close': 'btc_price'}))

# total = (total.merge(
#     market_data[['user_created_date_time', 'close']], how='left', on='user_created_date_time'
#         ).rename(columns={'close': 'btc_price'}))


3190


In [56]:
__total = (
        __total
        .groupby('user_id')
        .agg({'pnl': sum, 'volume': sum})
        .reset_index()
        .rename(columns={'pnl': 'full_pnl', 'volume': 'full_volume'})
        )

total = total.merge(__total, how='left', on='user_id')
#total = total.rename(columns={'pnl_x': 'pnl', 'pnl_y': 'full_pnl'})

In [57]:
total.tail()

,user_id,pnl,vol_sum,vol_mean,vol_median,trades_qty,account_qty,user_created_date_time,trade_time,deposit,withdrawal,first_dep,first_withd,withd_cond,dif_dep_with,diff_reg_trade,btc_price,full_pnl,full_volume
3185,xdark21,-13.83,"1,179.17",589.59,589.59,2,1,2019-08-20,0.95,NaN,NaN,NaN,NaN,NaN,0.00,1,"10,763.23",-13.83,"1,179.17"
3186,xpk,-461.95,"15,460.65","7,730.33","7,730.33",2,1,2019-04-02,41.53,"1,293.15","2,202.53","1,293.15",538.84,1.00,754.31,1,"4,879.88",-461.95,"15,460.65"
3187,xwerewolf,3.03,"6,587.77","3,293.88","3,293.88",2,1,2019-04-02,0.08,156.58,0.00,156.58,NaN,NaN,0.00,1,"4,879.88",-126.31,"30,979.90"
3188,zavodchik,-34.70,592.84,148.21,139.54,4,1,2020-04-13,157.35,143.71,124.87,7.12,7.73,1.00,-0.62,1,"6,845.04",-34.70,592.84
3189,zx007,-81.48,"53,163.22","1,208.26","1,259.98",44,1,2020-04-18,1.93,302.85,191.50,2.85,191.50,1.00,-188.65,1,"7,257.67",-77.98,"72,829.10"


In [58]:
total.isna().sum()

user_id                      0
pnl                          0
vol_sum                      0
vol_mean                     0
vol_median                   0
trades_qty                   0
account_qty                  0
user_created_date_time       0
trade_time                 163
deposit                     51
withdrawal                  51
first_dep                   53
first_withd               1110
withd_cond                1110
dif_dep_with                 0
diff_reg_trade               0
btc_price                    0
full_pnl                     0
full_volume                  0
dtype: int64

In [59]:
# filling na and drop with a lot of nan values
total = total.drop(columns={'first_withd', 'withd_cond'})
#total = total.fillna(total.median())
total = total.fillna({'deposit': total['deposit'].median(),
                      'trade_time': total['trade_time'].median(),
                      'withdrawal': total['withdrawal'].median(),
                      'first_dep': total['first_dep'].median()})

# encoding user_created_date_time
total['reg_year'] = pd.DatetimeIndex(total['user_created_date_time']).year
total['reg_month'] = pd.DatetimeIndex(total['user_created_date_time']).month
total['reg_day'] = pd.DatetimeIndex(total['user_created_date_time']).day
total = total.drop(columns={'user_created_date_time'})

In [60]:
from feature_engine.creation import CyclicalFeatures

cyclical = CyclicalFeatures(variables=None, drop_original=True)
dates = cyclical.fit_transform(total[['reg_month', 'reg_day']])
total = pd.concat([total, dates],axis=1)
total = total.drop(columns={'reg_month', 'reg_day'})
total.to_csv('data/input/dataset.csv', index=False)
total.tail()

,user_id,pnl,vol_sum,vol_mean,vol_median,trades_qty,account_qty,trade_time,deposit,withdrawal,...,dif_dep_with,diff_reg_trade,btc_price,full_pnl,full_volume,reg_year,reg_month_sin,reg_month_cos,reg_day_sin,reg_day_cos
3185,xdark21,-13.83,"1,179.17",589.59,589.59,2,1,0.95,288.28,53.87,...,0.00,1,"10,763.23",-13.83,"1,179.17",2019,-0.87,-0.50,-0.79,-0.61
3186,xpk,-461.95,"15,460.65","7,730.33","7,730.33",2,1,41.53,"1,293.15","2,202.53",...,754.31,1,"4,879.88",-461.95,"15,460.65",2019,0.87,-0.50,0.39,0.92
3187,xwerewolf,3.03,"6,587.77","3,293.88","3,293.88",2,1,0.08,156.58,0.00,...,0.00,1,"4,879.88",-126.31,"30,979.90",2019,0.87,-0.50,0.39,0.92
3188,zavodchik,-34.70,592.84,148.21,139.54,4,1,157.35,143.71,124.87,...,-0.62,1,"6,845.04",-34.70,592.84,2020,0.87,-0.50,0.49,-0.87
3189,zx007,-81.48,"53,163.22","1,208.26","1,259.98",44,1,1.93,302.85,191.50,...,-188.65,1,"7,257.67",-77.98,"72,829.10",2020,0.87,-0.50,-0.49,-0.87


In [61]:
len(total.columns)

21